<a href="https://colab.research.google.com/github/changsksu/IMSE_Data_Science/blob/main/NN_Control_Charts_for_Mean_shifts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NN Control Chart for detecting mean shifts
This NN has 10 input nodes, 2 hidden layers, and binary node. The output of 0 means the process is in control and 1 for out of control. A totla of 12000 N(0,1) is used as the in-control training set. The out of control data sets are 4000 N(1,1), 4000 N(2,1), and 4000 N(3,1) respectively. Using 80/20 splits, the testing data set contains IC data: 3000 N(0,1) and 3000 OOC data: 1000 N(mu,1) where mu=1,2,3

By Dr. Shing Chang, 2/22/2024 revised 3/25/2024


In [ ]:
# generate train and testing data sets
# in-control data set X N(0,1) 12000
# OOC data sets are
# small mean shift N(1,1) 4000
# medium mean shift N(2,1) 4000
# large mean shift N(3,1) 4000
# n=10 so the sample size = total obs/n
import numpy as np

# Set the mean and standard deviation for in
mean = 0
std_dev = 1

# Generate 12,000 normally distributed data points
data = np.random.normal(mean, std_dev, 12000)
dataICtrain = np.reshape(data, (1200, 10))

data = np.random.normal(mean, std_dev, 3000)
dataICtest = np.reshape(data, (300, 10))
#dataIC
# Print the first few data points as an example
#print(dataIC[:10])
#dataICtrain.shape
#dataICtest.shape
#
#
# Generate 400 small shift samples for training 100 for testing
mean = 1
std_dev = 1
data = np.random.normal(mean, std_dev, 4000)
dataOC1train = np.reshape(data, (400, 10))
data = np.random.normal(mean, std_dev, 1000)
dataOC1test = np.reshape(data, (100, 10))

# Generate 400 medium shift samples for training 100 for testing (n=10)
mean = 2
std_dev = 1
data = np.random.normal(mean, std_dev, 4000)
dataOC2train = np.reshape(data, (400, 10))
data = np.random.normal(mean, std_dev, 1000)
dataOC2test = np.reshape(data, (100, 10))

# Generate 400 large shift samples for training 100 for testing (n=10)
mean = 3
std_dev = 1
data = np.random.normal(mean, std_dev, 4000)
dataOC3train = np.reshape(data, (400, 10))
data = np.random.normal(mean, std_dev, 1000)
dataOC3test = np.reshape(data, (100, 10))


In [ ]:
# compute x bars and stds for the IC training data
# then compute x bar bar and s bar to estimate the overall mean and std of the training process
x_bars = np.mean(dataICtrain, axis=1)
stds = np.std(dataICtrain, axis=1)
x_2bar =np.mean(x_bars)
s_bar =np.mean(stds)
x_2bar

-0.014589687323112287

In [ ]:
# # Apply Levine transformation for all datasets
# this step is to standardize any dataset (mu, sigma)
# all negative data is flipped to the positve side
ICtrain = np.abs(dataICtrain - x_2bar) / s_bar
ICtest = np.abs(dataICtest - x_2bar) / s_bar
OC1train = np.abs(dataOC1train - x_2bar) / s_bar
OC1test = np.abs(dataOC1test - x_2bar) / s_bar
OC2train = np.abs(dataOC2train - x_2bar) / s_bar
OC2test = np.abs(dataOC2test - x_2bar) / s_bar
OC3train = np.abs(dataOC3train - x_2bar) / s_bar
OC3test = np.abs(dataOC3test - x_2bar) / s_bar
# np.mean(OC3train, axis=0)

In [ ]:
# Concatenate matrices along rows (vertically) of all training data sets
Xtraining = np.concatenate((ICtrain, OC1train, OC2train, OC3train), axis=0)
X_train= np.asarray(Xtraining)
Xtesting = np.concatenate((ICtest, OC1test, OC2test, OC3test), axis=0)
X_test= np.asarray(Xtesting)

In [ ]:
# generate target response data
# training target Y
ICtarget =np.full(1200, 0)
OC1target=np.full(400,1)
OC2target=np.full(400,1)
OC3target=np.full(400,1)
Y_train=np.concatenate((ICtarget, OC1target, OC2target, OC3target), axis=0)
#
# testing target Y
ICtarget =np.full(300, 0)
OC1target=np.full(100,1)
OC2target=np.full(100,1)
OC3target=np.full(100,1)
Y_test=np.concatenate((ICtarget, OC1target, OC2target, OC3target), axis=0)

In [ ]:
Y_test.shape

(600,)

# Backward Propagration Training:
**Use Keras to define a Neural network that will be trained off of this data. This Neural Network can then be used to predict sample (size n=10) is in control or out of control.**
ref. https://keras.io/guides/sequential_model/

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential() # Initialising the ANN

classifier.add(Dense(units = 10, activation = 'relu', input_dim=10))
classifier.add(Dense(units = 20, activation = 'relu'))
classifier.add(Dense(units = 20, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

Specify the optimzer and loss function.

In [ ]:
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy')

You now train the neural network using Classifier.fit, passing it the training data -- i.e. for this set of X, this is what the Y should look like. The NN will then spot the patterns in the data, and build a neural network that could replicate that.

In [ ]:
# try 10 epochs to improve the performance
classifier.fit(X_train, Y_train, batch_size = 1, epochs = 10)

Epoch 1/10
2400/2400 [==============================] - 9s 3ms/step - loss: 0.4380
Epoch 2/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.2874
Epoch 3/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.2816
Epoch 4/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.2793
Epoch 5/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.2833
Epoch 6/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.2705
Epoch 7/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.2751
Epoch 8/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.2755
Epoch 9/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.2754
Epoch 10/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.2769


After 10 epochs of train, we only achieve about 72% accuracy.

In [ ]:
# try 100 epochs to improve the performance
# classifier.fit(X_train, Y_train, batch_size = 1, epochs = 100)

**Your turn: does the increasing training epochs improve the NN performance?**

# Feedforward NN Prediction:
To predict new values, the Neural Network uses classifier.predict. The test values for X (which the Neural Network hasn't previously seen) will give back a set of predictions. These predicitons will be probabilities, if thye are greater than .5, the process is deemed out of control. Otherwise, the process is deemed in control.

In [ ]:
Y_pred = classifier.predict(X_test)
Y_pred = [ 1 if y>=0.5 else 0 for y in Y_pred ]

19/19 [==============================] - 0s 2ms/step


Now we can loop through the set of predicitons for the test set (called Y_pred) and the actual values for the test set (celled Y_test), and see how alike they are -- if they are the same, I'll increment 'correct', otherwise I'll incrememnt 'wrong'.

You'll see the result is 100% accurate, even though the neural network reported a lower accuracy than that. Why?

In [ ]:
total = 0
correct = 0
wrong = 0
for i in range(len(Y_pred)):
  total=total+1
  if(Y_test[i] == Y_pred[i]):
    correct=correct+1
  else:
    wrong=wrong+1

print("Total " + str(total))
print("Correct " + str(correct))
print("Wrong " + str(wrong))

Total 600
Correct 539
Wrong 61


In [ ]:
correct/total

0.8983333333333333

**Your Turn:** what are the other ways to access the false positive and false negative rates? In the SQC literature, false positve is called Type I error where the false negative is called Typle II error (assuming Positive is process is out of control and Negative is process is in control).

In [ ]:
# to use the trained NN for monitoring the process
# for examle, a sample of 10 with 1.5 sigma shift
mean = 0.7
std_dev = 1
sample1 = np.random.normal(mean, std_dev, 10)
transform1 = np.abs(sample1 - x_2bar) / s_bar
X_data= np.asmatrix(transform1)
outcome1=classifier.predict(X_data)
# print(outcome1)
print("Prediction is ",outcome1)
if outcome1>=0.5:
    print("The process is now out of control")
else:
    print("The process is in control")



# Your Turn: try various (mean, std_dev) combinations, answer the following questions:
1. Does this NN senstive to small process mean shifts?
2. Does this NN response to process variance changes?
3. How do you improve this NN? (e.g. add sample mean and std as input, more hidden layers, more training data, bigger n, longer training time, etc,)
4. Advanced: try CNN and use (time , data, prior trend) as the input where prior trend is the distance away from the target from the previous outcome or CUSUM)